<font color='green'> 
**Kodluyoruz - Data Science for Public Good (28.03.2020-20.06.2020)**

İBB iş birliğiyle yürütülen programda İBB'den aldığımız **trafik duyuruları** ve **trafik yoğunluğu** veri setinden faydalanarak trafik yoğunluğunu tahmin eden model geliştirdik. 
</font>

### İzlediğimiz Adımlar

**1. Kütüphanelerin Yüklenmesi**

**2. Verinin Okunması**

**3. Verinin İncelenmesi**

  * Aynı konumda saat aralıklarına bakıyoruz.
  * sxk9u0 konumunun sınırda olduğundan hem Kağıthane hem Beşiktaş olarak verisinin olduğunu farkettik. Bunun gibi olan başka konum var mı kontrol ediyoruz.
  * "Trafik yoğunluk" ve "tkm duyuru" verisinde bulunan geohashlerin eşleşme durumlarını ve birbirinden farklı geohashlerini tespit etmek için kod yazıyoruz.

**4. Data setlerinin birleştirilmesi**

* Traffic density ve TKM data framelerini birleştirmek için ön işlemlerin yapılması
  * Trafik yoğunluk verileri ile TKM verilerinde sütun isimlerinin düzenlenmesi ve zamana göre sıralanması
  * TKM - TIME sütunu saniyelerin sıfırlanması
  * TIME sutunlarının sıralanması
  * Farklı İki İlçede Aynı Geohashe Düşen Değerlerin Ortalamasının Alınması

* Traffic density ve TKM data framelerinin birleştirilmesi

**5. Feature Engineering Yapılması**

* Merge edilmiş data setinde "TIME" columnundan anlamlı feature'lar çıkarılması
  * TIME columnundan başka bilgiler çıkarabilmek için fonksiyon yazdık.
  * Tarihin haftanın hangi gününe denk geldiği bilgisini ekledik.
  * Tarihin yılın hangi ayına denk geldiği bilgisini ekledik.
  * Saatin öğleden önce mi sonra mı olduğu bilgisini ekledik.
  * Tarihin hafta sonuna denk gelip gelmediği bilgisini ekledik.
  * İş Saatlerinin Eklenmesi
  * Mevsim Sütunu Eklenmesi
  * Resmi Tatil Sütunu Eklenmesi
  * is_pandemic sutunu eklenmesi
  * Okul Sütunu Eklenmesi
  * Geohash6 ile Time sütununun birleştirilerek index olarak atanması

Dosyanın Bu Halinin Feather ile Kaydedilmesi
* Feature'ların Nan Değerlerinin Doldurulması
* Modele Girecek Feature'ların Belirlenmesi
* Modele Girecek Feature'ların Nümerik Değerlere Çevrilmesi
* Tüm Feature'lar İçin One-hot Encoding Yapılması

Dosyanın Bu Halinin Feather ile Kaydedilmesi
* Hedef Değişken Olarak Belirlediğimiz Ortalama Hızın Sınıflandırma Algoritması İçin Düzenlenmesi

Dosyanın Bu Halinin Feather ile Kaydedilmesi

**6. Model Kurma**

Kullanılan Algoritmalar
* Random Forest
* XGBoost
* Logistic Regression

**7. y değerinin belirlenmesi**

**8. Görselleştirme Çalışması**

* Trafik Yoğunluk Verisiyle Grafik Çalışmaları
* TKM Verisindeki Kooordinat Verisinden Yararlanılarak Harita Görselleştirme Çalışması

### <font color='blue'/> **1. Kütüphanelerin Yüklenmesi**  </font>

In [ ]:
pip install pygeohash 

In [ ]:
import pandas as pd
import numpy as np
import pygeohash as pgh 
import seaborn as sns 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os

In [ ]:
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Data")

### <font color='blue'/> **2. Verinin Okunması**  </font>

**TKM Duyuru Data seti**

In [ ]:
tkm=pd.read_excel("TKM.xlsx", index_col=0, encoding='utf-8')

In [ ]:
tkm.shape

In [ ]:
tkm.head()

In [ ]:
df_tkm=tkm.copy()

In [ ]:
df_tkm.drop(["GEOHASH_CHECK","TRAFIK_DENSITY_Geohash"], axis=1, inplace=True) 

In [ ]:
df_tkm.head()

In [ ]:
df_tkm["ANNOUNCE_TYPE"].value_counts()

In [ ]:
df_tkm[df_tkm["ANNOUNCE_TYPE"]=="HABER"]

In [ ]:
df_tkm['COUNTY_NAME'].value_counts()

**Traffic Density veri seti**

In [ ]:
traffic=pd.read_json("TRAFFIC_DENSITY.json")

In [ ]:
traffic.shape

In [ ]:
traffic.head()

In [ ]:
df_traffic=traffic.copy()

In [ ]:
df_traffic['COUNTY_NAME'].value_counts()

In [ ]:
df_traffic.isnull().sum()

### <font color='blue'/> **3. Verinin İncelenmesi**  </font>

#### <font color='black'/> **Aynı konumda saat aralıklarına bakıyoruz.** </font> 

In [ ]:
df_traffic[df_traffic['GEOHASH6']=="sxk9u0"].head(10).sort_values(["DATE_TIME"])

#### <font color='black'/> **sxk9u0 konumunun sınırda olduğundan hem Kağıthane hem Beşiktaş olarak verisinin olduğunu farkettik. Bunun gibi olan başka konum var mı kontrol ediyoruz.** </font> 

In [ ]:
df_besiktas=df_traffic[df_traffic['COUNTY_NAME']=='BEŞİKTAŞ'].reset_index()  
# reset_index() demeseydim indeksleri karışık bırakıyordu, aşağıda yazdığım for döngüsünde range(len..) kısmında sıkıntı çıkartıyordu.
df_besiktas

In [ ]:
df_kagithane=df_traffic[df_traffic['COUNTY_NAME']=='KAĞITHANE'].reset_index()
df_kagithane

In [ ]:
l=[]

for i in range(len(df_besiktas)):
    l.append(df_besiktas['GEOHASH6'][i])  # Beşiktaş'taki geohashleri listeye attım. 
    
s_b=set(l) # Listeyi set'e çevirerek geohashleri bir kez almış oldum. 
# s_b

In [ ]:
m=[]

for i in range(len(df_kagithane)):
    m.append(df_kagithane['GEOHASH6'][i])
    
s_k=set(m)
# s_k

In [ ]:
list_besiktas=list(s_b)  # oluşturduğum setleri aşağıda for döngüsüne sokabilmek için tekrar listeye çevirmek durumunda kaldım.
list_kagithane=list(s_k)

In [ ]:
p=[]

for i in range(len(list_besiktas)):
    for j in range(len(list_kagithane)):
        if list_besiktas[i]==list_kagithane[j]:  
            p.append(list_besiktas[i])
 # Beşiktaş'ın geohash listesinde bulunan elemanlar Kağıthane'de de bulunuyorsa o elemanları bir listeye ekle dedim.           
p

#### <font color='black'/> **"Trafik yoğunluk" ve "tkm duyuru" verisinde bulunan geohashlerin eşleşme durumlarını ve birbirinden farklı geohashlerini tespit etmek için aşağıdaki kodu yazdık.** </font> 

In [ ]:
traffic_geohash=df_traffic['GEOHASH6']  # Trafik datasındaki geohash sütununu çektim.
traffic_geohash

In [ ]:
list_traffic=list(traffic_geohash.values)
s_traffic=set(list_traffic)

In [ ]:
len(s_traffic)

In [ ]:
tkm_geohash=df_tkm['ANNOUNCE_GEOHASH6']     # Tkm datasındaki geohash sütununu çektim.
tkm_geohash

In [ ]:
list_tkm=list(tkm_geohash.values)
s_tkm=set(list_tkm)

In [ ]:
len(s_tkm)

In [ ]:
traffic_unique_list=list(s_traffic)   # oluşturduğum setleri aşağıda for döngüsüne sokabilmek için listeye çevirdim.
tkm_unique_list=list(s_tkm)

In [ ]:
same_geohash_list=[]

for i in range(len(traffic_unique_list)):
    for j in range(len(tkm_unique_list)):
        if traffic_unique_list[i]==tkm_unique_list[j]: # Hem trafik datasında bulunan hem de tkm datasında bulunan geohashleri listeye ekledim.
            same_geohash_list.append(traffic_unique_list[i])
            
len(same_geohash_list) # İkisinde de olan geohashlerin sayısı 134'müş.

In [ ]:
exist_traffic_geohash=traffic_unique_list.copy()
for e in same_geohash_list:
    exist_traffic_geohash.remove(e)  # trafik datasında olan ama tkm datasında olmayan geohashleri buldum.
    
len(exist_traffic_geohash) # 152-134=18 

In [ ]:
exist_tkm_geohash=tkm_unique_list.copy()
for e in same_geohash_list:
    exist_tkm_geohash.remove(e) # tkm datasında olan ama trafik datasında olmayan geohashleri buldum.
    
len(exist_tkm_geohash)

### <font color='blue'/> **4. Data setlerinin birleştirilmesi** </font>

#### <font color='black'/> **Traffic density ve TKM data framelerini birleştirmek için ön işlemlerin yapılması** </font>

##### <font color='brown'/> **Trafik yoğunluk verileri ile TKM verilerinde sütun isimlerinin düzenlenmesi ve zamana göre sıralanması** </font>

In [ ]:
df_traffic.rename(columns={'DATE_TIME':'TIME'}, inplace=True) 
df_traffic.columns  # Tarihe ve geohash kodlarına göre verileri birleştireceğimiz için sütun isimlerini eşitliyoruz

In [ ]:
df_tkm.rename(columns={'ANNOUNCE_TIME':'TIME','ANNOUNCE_GEOHASH6':'GEOHASH6'}, inplace=True)
df_tkm.columns

In [ ]:
df_traffic.sort_values(by=['TIME','GEOHASH6'],inplace=True)
df_tkm.sort_values(by=['TIME','GEOHASH6'],inplace=True)

##### <font color='brown'/> **TKM - TIME sütunu saniyelerin sıfırlanması**  </font>

TKM veri setinde TIME sutunu: (Yıl /ay /gün) - (saat / dakika / saniye) şeklinde tutuluyor. merge tolerance problemini çözmek için saniyeleri 0 lamak istiyoruz. Bunun için formattan saniyeyi kaldırabilir, aynı şeyi traffic density dataframe'ine de uygulayabiliriz.

In [ ]:
# bu ne zaman gerekli bak import datetime 

In [ ]:
df_tkm['TIME']

In [ ]:
df_tkm['TIME'] = pd.to_datetime(df_tkm['TIME'])
df_tkm['TIME'] = df_tkm['TIME'].apply(lambda t: t.replace(second=0))

In [ ]:
type(df_tkm.loc[1,"TIME"])  

In [ ]:
type(df_traffic.loc[1,"TIME"])

In [ ]:
df_tkm['TIME']

##### <font color='brown'/> **TIME sutunlarını sıraladık.** </font>  

In [ ]:
df_traffic.sort_values(by=['TIME','GEOHASH6'],inplace=True)
df_tkm.sort_values(by=['TIME','GEOHASH6'],inplace=True)

In [ ]:
df_traffic.tail()

In [ ]:
df_tkm.tail()

##### <font color='brown'/> **Farklı İki İlçede Aynı Geohashe Düşen Değerlerin Ortalamasının Alınması.** </font> 

In [ ]:
df_traffic

In [ ]:
#kesişimde kalan geohashlerin indexlerini filter_1 e atadik. yani istediğimiz indexleri filter 1 tutuyor.
intersect_geo=['sxk9sp', 'sxk9sn', 'sxk9u0', 'sxk9u1', 'sxk9sj']
filter_1 = df_traffic["GEOHASH6"].isin(intersect_geo)

In [ ]:
filter_1

In [ ]:
#traffic yoğunluğu df'sinde, istedigimiz geohashler ile kesişen satirlari  vals  df sine atadik. 
vals = df_traffic[filter_1].groupby(['TIME', "GEOHASH6","COUNTY_NAME"], as_index=False)[['VEHICLE_COUNT','AVG_SPEED']].mean()

In [ ]:
vals

In [ ]:
df_traffic.drop(df_traffic[filter_1].index, inplace = True)
# kesişen satırların hepsini kaldırıyoruz daha sonra ortalamasını aldığımız satırları ekleyeceğiz

In [ ]:
df_traffic

In [ ]:
vals.replace(regex=['KAĞITHANE'], value='BEŞİKTAŞ',inplace=True)

In [ ]:
vals

In [ ]:
vals_1 = vals.groupby(['TIME', "GEOHASH6","COUNTY_NAME"], as_index=False)[['VEHICLE_COUNT','AVG_SPEED']].mean()

In [ ]:
vals_1

In [ ]:
df_traffic=df_traffic.append(vals_1, sort=False)

In [ ]:
df_traffic

In [ ]:
df_traffic.sort_values(by=['TIME','GEOHASH6'],inplace=True)

In [ ]:
df_traffic

#### <font color='black'/> **Traffic density ve TKM data framelerinin birleştirilmesi** </font>

In [ ]:
df_tkm.columns  

In [ ]:
df_traffic.columns

In [ ]:
data=pd.merge_asof(left=df_traffic,right=df_tkm,by=['GEOHASH6'], on='TIME',direction='nearest',tolerance=pd.Timedelta('15minute'))
data.head(100)

In [ ]:
data.isnull().sum()

In [ ]:
data.tail()

### <font color='blue'/> **5. Feature Engineering Yapılması** </font>

#### <font color='black'/> **Merge edilmiş data setinde "TIME" columnundan anlamlı feature'lar çıkarılması** </font>

##### <font color='brown'/> **TIME columnundan başka bilgiler çıkarabilmek için fonksiyon yazdık.**  </font>

In [ ]:
def add_datepart(df, dt_name):
# drop=True ekleyip aşağıdaki if'i yazabiliriz sadece True dediğimizde eklesin istiyorsak
 
    dt_column = df[dt_name]
    
    attr = ['Year', 'Week', 'Day','Hour', 'Minute','Dayofyear','Date',
            'Is_month_end', 'Is_month_start','Is_quarter_start','Is_quarter_end', 'Is_year_end', 'Is_year_start']
    
    for a in attr: 
        
        df[a] = getattr(dt_column.dt, a.lower())
        
   
    #if drop: 
        #df.drop(dt_name, axis=1, inplace=True)

In [ ]:
add_datepart(data,"TIME")

In [ ]:
data.tail()

##### <font color='brown'/> **Tarihin haftanın hangi gününe denk geldiği bilgisini ekledik.** </font>

In [ ]:
data['DayofWeek'] = data['TIME'].dt.strftime('%A')
data.head()

##### <font color='brown'/> **Tarihin yılın hangi ayına denk geldiği bilgisini ekledik.** </font>

In [ ]:
data['Month'] = data['TIME'].dt.strftime('%B')
data.tail()

##### <font color='brown'/> **Saatin öğleden önce mi sonra mı olduğu bilgisini ekledik.** </font>

In [ ]:
data['AM/PM'] = data['TIME'].dt.strftime('%p')
data.tail()

##### <font color='brown'/> **Tarihin hafta sonuna denk gelip gelmediği bilgisini ekledik.** </font>

In [ ]:
data['is_weekend'] = ((data["TIME"].dt.dayofweek) // 5 == 1).astype(int)
data.tail()

##### <font color='brown'/>**İş Saatlerinin Eklenmesi**</font>

In [ ]:
#set index işlemi yaparken TIME sütunu gitmesin diye kopyalanan sütun üzerinden işlem yapılacak
data["copy_TIME"]=data["TIME"].copy() 

In [ ]:
data.set_index('copy_TIME',inplace=True)

In [ ]:
rush_morning=data.between_time('06:00:00', '09:00:00')
rush_evening=data.between_time('17:00:00', '20:00:00')

In [ ]:
rush_hours_list=list(rush_morning.index.values.tolist())+list(rush_evening.index.values.tolist())

In [ ]:
data['rush_hours'] = 0          
data.loc[data.index.isin(rush_hours_list), 'rush_hours'] = 1 

In [ ]:
data[data['rush_hours'] == 1] 

In [ ]:
data.reset_index(drop=True, inplace=True)

In [ ]:
data.head()

##### <font color='brown'/>**Mevsim Sütunu Eklenmesi**  </font>

In [ ]:
from datetime import datetime

In [ ]:
Y=2020

seasons = {'Summer':(datetime(Y,6,21), datetime(Y,9,22)),
           'Autumn':(datetime(Y,9,23), datetime(Y,12,20)),
           'Spring':(datetime(Y,3,21), datetime(Y,6,20))}


def get_season(date):
    for season,(season_start, season_end) in seasons.items():
        if date>=season_start and date<= season_end:
            return season
    else:
        return 'Winter'

data['Season']= data['TIME'].map(get_season)

In [ ]:
print("When the date is", data['TIME'].loc[1000], "and the season is", data['Season'].loc[1000])

print("When the date is", data['TIME'].loc[1050000], "and the season is", data['Season'].loc[1050000])

print("When the date is", data['TIME'].loc[1050000], "and the season is", data['Season'].loc[1050000])

print("When the date is", data['TIME'].loc[1552430], "and the season is", data['Season'].loc[1552430])

In [ ]:
data.head()

##### <font color='brown'/>**Resmi Tatil Sütunu Eklenmesi**</font>

In [ ]:
df_public_holiday=pd.read_excel('ResmiTatiller.xlsx')

In [ ]:
df_public_holiday

In [ ]:
holidays=df_public_holiday["Tarih"].values.tolist()

In [ ]:
data['is_holiday']=data['TIME'].dt.date.astype('datetime64').isin(holidays)

In [ ]:
data['is_holiday']

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.loc[1570000]

In [ ]:
data.loc[1475800]

##### <font color='brown'/>**is_pandemic sutunu eklenmesi** </font> 

In [ ]:
data["TIME"]=pd.to_datetime(data["TIME"]) 

In [ ]:
pandemic_start_date="2020.03.11"
pandemic_end_date="2020.06.19"

In [ ]:
pandemic_dates=(data["TIME"]>= pandemic_start_date) & (data["TIME"] <= pandemic_end_date)

In [ ]:
pandemic_dates_list=data[pandemic_dates].index.values.tolist()

In [ ]:
data['is_pandemic'] = 0          
data.loc[data.index.isin(pandemic_dates_list), 'is_pandemic'] = 1 

In [ ]:
data[data['is_pandemic'] == 1] 

##### <font color='brown'/>**Okul Sütunu Eklenmesi**</font> 

In [ ]:
first_term_start_date="2019.09.09"
first_term_end_date="2020.01.17"

second_term_start_date="2020.02.03"
second_term_end_date="2020.03.15" # pandemiden dolayı 16 Mart itibariyle tatil edilmişti. 

In [ ]:
data["TIME"]=pd.to_datetime(data["TIME"]) 

In [ ]:
first_term=(data["TIME"]>= first_term_start_date) & (data["TIME"] <= first_term_end_date)
second_term=(data["TIME"]>= second_term_start_date) & (data["TIME"] <= second_term_end_date)

In [ ]:
school_dates_list=list(data[first_term].index.values.tolist())+list(data[second_term].index.values.tolist())

In [ ]:
data['is_school'] = 0          
data.loc[data.index.isin(school_dates_list), 'is_school'] = 1 

In [ ]:
data[data['is_school'] == 0]   

In [ ]:
data.head()

##### <font color='brown'/> **Geohash6 ile Time sütununun birleştirilerek index olarak atanması**</font> 

In [ ]:
data["combined"]=data.apply(lambda x:"%s_%s" % (x["GEOHASH6"],x["TIME"]),axis=1)  # %s birleştir ve tek dize yap anlamında.

In [ ]:
data.head()

In [ ]:
data.set_index('combined',inplace=True)

In [ ]:
data.loc["sxk93r_2020-01-06 09:30:00"]

In [ ]:
data.head()

#### <font color='green'/>**Dosyanın Bu Halinin Feather ile Kaydedilmesi**  </font> 

In [ ]:
import os

In [ ]:
data=data.reset_index()

In [ ]:
os.makedirs('tmp', exist_ok=True) # Don't give error if file is already there

data.to_feather('tmp/raw')

#### <font color='black'/> **Feature'ların Nan Değerlerinin Doldurulması** </font>

In [ ]:
raw = pd.read_feather("tmp/raw")

In [ ]:
raw.isnull().sum()

##### <font color='brown'/>**ANNOUNCE_TYPE sütununda Nan değerlerin "No Announce" ile doldurulması**  </font> 

In [ ]:
raw["ANNOUNCE_TYPE"].fillna("No Announce", inplace = True)

In [ ]:
raw["ANNOUNCE_TYPE"].value_counts()

In [ ]:
raw.set_index('combined',inplace=True)

#### <font color='black'/> **Modele Girecek Feature'ların Belirlenmesi**</font> 

In [ ]:
df_a=raw.drop("TIME", axis=1) 
# time sütunundan zaman bilgileri ile yeni feature lar üretmiştik.
# Tıme ı atıyoruz hata veriyor atmazsak. 
# Asıl data frame'de kalsın diye başka df adıyla kaydettik. 

In [ ]:
df_a.columns

In [ ]:
model_list=['COUNTY_NAME_x', 'AVG_SPEED', 'VEHICLE_COUNT','ANNOUNCE_TYPE', 'DayofWeek', 'Month', 'AM/PM', 'is_weekend',
        'Hour', 'Is_month_end',
       'Is_month_start', 'Is_quarter_start', 'Is_quarter_end',
       'Is_year_start', 'rush_hours', 'Season', 'is_holiday','is_pandemic','is_school']

In [ ]:
df_a = df_a.reindex(columns=model_list)

In [ ]:
len(model_list)

In [ ]:
df = df_a[model_list]

In [ ]:
df.head()

In [ ]:
df.isna().sum()

#### <font color='black'/> **Modele Girecek Feature'ların Nümerik Değerlere Çevrilmesi**</font>

In [ ]:
df["AM/PM"]=df["AM/PM"].replace({"AM":1,"PM":0}) 

In [ ]:
df.rename(columns={"AM/PM": "is_AM"}, inplace=True) 

In [ ]:
df.head()

In [ ]:
boolean_cols = ["Is_month_end","Is_month_start","Is_quarter_start","Is_quarter_end","Is_year_start","is_holiday","is_pandemic","is_school"]
for boolean_col in boolean_cols:
  df[boolean_col] = df[boolean_col].replace({False: 0, True:1})

In [ ]:
df.info()

#### <font color='black'/> **Tüm Feature'lar İçin One-hot Encoding Yapılması**</font>

In [ ]:
pd.get_dummies(df).head()

In [ ]:
x = pd.get_dummies(df)
for col in x.columns:
         print(x[col].value_counts())

#### <font color='green'/>**Dosyanın Bu Halinin Feather ile Kaydedilmesi**  </font> 

In [ ]:
x=x.reset_index()

In [ ]:
import os

os.makedirs('tmp', exist_ok=True) # Don't give error if file is already there
x.to_feather('tmp/one_hot_encoding')

#### <font color='black'/> **Hedef Değişken Olarak Belirlediğimiz Ortalama Hızın Sınıflandırma Algoritması İçin Düzenlenmesi**</font>

In [ ]:
df_h = pd.read_feather('tmp/one_hot_encoding')

In [ ]:
df_h.head()

In [ ]:
df_h.set_index('combined',inplace=True)

In [ ]:
df=df_h.copy()

In [ ]:
raw = pd.read_feather("tmp/raw")

In [ ]:
raw[raw["ANNOUNCE_TYPE"]=="YOĞUN TRAFIK"]["AVG_SPEED"].median()

In [ ]:
treshold=raw[raw["ANNOUNCE_TYPE"]=="YOĞUN TRAFIK"]["AVG_SPEED"].median()

def is_traffic(speed):

    if speed<=treshold:
       return 1 
    else:
       return 0

In [ ]:
df["is_traffic"]= df['AVG_SPEED'].map(is_traffic)

In [ ]:
df.head()

#### <font color='green'/>**Dosyanın Bu Halinin Feather ile Kaydedilmesi**  </font> 

In [ ]:
df=df.reset_index()

In [ ]:
import os

os.makedirs('tmp', exist_ok=True) 
df.to_feather('tmp/df_classification')

### <font color='blue'/> **6. Model Kurma** </font>

**Kullanılan Algoritmalar**

1.   Random Forest
2.   XGBoost
3.   Logistic Regression

In [ ]:
df_c=pd.read_feather('tmp/df_classification')

In [ ]:
df_c.set_index('combined',inplace=True)

In [ ]:
df_c.head()

#### <font color='green'/> **Random Forest** </font>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix as cm
import matplotlib.pyplot as plt

##### <font color='brown'/> **Sadece avg_speed'i attım 100.000 veriyle çalıştırdım.** </font> 

In [ ]:
model_list_real=['combined','AVG_SPEED','VEHICLE_COUNT', 'is_AM', 'is_weekend',
        'Hour', 'Is_month_end',
       'Is_month_start', 'Is_quarter_start', 'Is_quarter_end',
       'Is_year_start', 'rush_hours', 'Season', 'is_holiday',
       'is_pandemic','is_school',
        'COUNTY_NAME_x_BEŞİKTAŞ', 'COUNTY_NAME_x_ESENLER','COUNTY_NAME_x_KAĞITHANE',
        'ANNOUNCE_TYPE_ALT YAPI ÇALIŞMASI',
       'ANNOUNCE_TYPE_ARAÇ ARIZASI', 'ANNOUNCE_TYPE_ARAÇ YANGINI',
       'ANNOUNCE_TYPE_BAKIM-ONARIM ÇALIŞMASI', 'ANNOUNCE_TYPE_HABER',
       'ANNOUNCE_TYPE_KAZA BILDIRIMI', 'ANNOUNCE_TYPE_No Announce',
       'ANNOUNCE_TYPE_YOLUN TRAFIĞE KAPANMASI', 'ANNOUNCE_TYPE_YOĞUN TRAFIK',
       'ANNOUNCE_TYPE_ÇEVRE DÜZENLEMESI','DayofWeek_Friday',
       'DayofWeek_Monday', 'DayofWeek_Saturday', 'DayofWeek_Sunday',
       'DayofWeek_Thursday', 'DayofWeek_Tuesday', 'DayofWeek_Wednesday',
       'Month_April', 'Month_February', 'Month_January', 'Month_June',
       'Month_March', 'Month_May', 'Season_Spring', 'Season_Winter','is_traffic'] 

In [ ]:
model_list_real1=['VEHICLE_COUNT', 'is_AM', 'is_weekend',
        'Hour', 'Is_month_end',
       'Is_month_start', 'Is_quarter_start', 'Is_quarter_end',
       'Is_year_start', 'rush_hours', 'is_holiday',
       'is_pandemic','is_school',
        'COUNTY_NAME_x_BEŞİKTAŞ', 'COUNTY_NAME_x_ESENLER','COUNTY_NAME_x_KAĞITHANE',
        'ANNOUNCE_TYPE_ALT YAPI ÇALIŞMASI',
       'ANNOUNCE_TYPE_ARAÇ ARIZASI', 'ANNOUNCE_TYPE_ARAÇ YANGINI',
       'ANNOUNCE_TYPE_BAKIM-ONARIM ÇALIŞMASI', 'ANNOUNCE_TYPE_HABER',
       'ANNOUNCE_TYPE_KAZA BILDIRIMI', 'ANNOUNCE_TYPE_No Announce',
       'ANNOUNCE_TYPE_YOLUN TRAFIĞE KAPANMASI', 'ANNOUNCE_TYPE_YOĞUN TRAFIK',
       'ANNOUNCE_TYPE_ÇEVRE DÜZENLEMESI','DayofWeek_Friday',
       'DayofWeek_Monday', 'DayofWeek_Saturday', 'DayofWeek_Sunday',
       'DayofWeek_Thursday', 'DayofWeek_Tuesday', 'DayofWeek_Wednesday',
       'Month_April', 'Month_February', 'Month_January', 'Month_June',
       'Month_March', 'Month_May', 'Season_Spring', 'Season_Winter','is_traffic'] 
       # model_listten AVG_SPEED'i attım. 

In [ ]:
df_c.head()

In [ ]:
df_c = df_c.reindex(columns=model_list_real1)

In [ ]:
df=df_c[model_list_real1]

In [ ]:
def split_train_test(df,n): 
    
    return df[:n].copy(), df[n:].copy()

In [ ]:
n_sample1 = 100000  
n_remain1 = len(df)-n_sample1
df_sample1, df_remain1 = split_train_test(df, n_sample1) # İlk 100.000'ini almış olduk. 

df_sample1.shape, df_remain1.shape

In [ ]:
n_sample1_train = int(len(df_sample1)*(3/5)) 
n_sample1_test = int(len(df_sample1)-n_sample1_train)

df_sample1_train, df_sample1_test = split_train_test(df_sample1, n_sample1_train)

x_sample1_train=df_sample1_train.drop('is_traffic', axis=1)
y_sample1_train = df_sample1_train["is_traffic"]

x_sample1_test=df_sample1_test.drop('is_traffic', axis=1)
y_sample1_test = df_sample1_test["is_traffic"]


df_sample1_train.shape, df_sample1_test.shape, x_sample1_train.shape, y_sample1_train.shape, x_sample1_test.shape, x_sample1_test.shape

In [ ]:
df.isnull().sum()

In [ ]:
m1 = RandomForestClassifier(n_jobs=-1) 
m1.fit(x_sample1_train,y_sample1_train)
predictions = m1.predict(x_sample1_test)
score = round(accuracy_score(y_sample1_test, predictions), 3)
cm1 = cm(y_sample1_test, predictions)
sns.heatmap(cm1, annot=True, fmt=".0f")
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.title('Accuracy Score: {0}'.format(score), size = 15)
plt.show()

##### <font color='brown'/>**Hem AVG_SPEED'i hem de Vehicle Count'ı attım. 100.000 veriyle çalıştırdım.** </font> 

In [ ]:
model_list_real2=['is_AM', 'is_weekend',
        'Hour', 'Is_month_end',
       'Is_month_start', 'Is_quarter_start', 'Is_quarter_end',
       'Is_year_start', 'rush_hours', 'is_holiday',
       'is_pandemic','is_school',
        'COUNTY_NAME_x_BEŞİKTAŞ', 'COUNTY_NAME_x_ESENLER','COUNTY_NAME_x_KAĞITHANE',
        'ANNOUNCE_TYPE_ALT YAPI ÇALIŞMASI',
       'ANNOUNCE_TYPE_ARAÇ ARIZASI', 'ANNOUNCE_TYPE_ARAÇ YANGINI',
       'ANNOUNCE_TYPE_BAKIM-ONARIM ÇALIŞMASI', 'ANNOUNCE_TYPE_HABER',
       'ANNOUNCE_TYPE_KAZA BILDIRIMI', 'ANNOUNCE_TYPE_No Announce',
       'ANNOUNCE_TYPE_YOLUN TRAFIĞE KAPANMASI', 'ANNOUNCE_TYPE_YOĞUN TRAFIK',
       'ANNOUNCE_TYPE_ÇEVRE DÜZENLEMESI','DayofWeek_Friday',
       'DayofWeek_Monday', 'DayofWeek_Saturday', 'DayofWeek_Sunday',
       'DayofWeek_Thursday', 'DayofWeek_Tuesday', 'DayofWeek_Wednesday',
       'Month_April', 'Month_February', 'Month_January', 'Month_June',
       'Month_March', 'Month_May', 'Season_Spring', 'Season_Winter','is_traffic'] # AVG_SPEED ve VEHICLE_COUNT'ı attım.

In [ ]:
df=df_c[model_list_real2]

In [ ]:
def split_train_test(df,n): 
    
    return df[:n].copy(), df[n:].copy()

In [ ]:
n_sample2 = 100000  
n_remain2 = len(df)-n_sample2
df_sample2, df_remain2 = split_train_test(df, n_sample2)

df_sample2.shape, df_remain2.shape

In [ ]:
n_sample2_train = int(len(df_sample2)*(3/5)) # test setimin boyutu :len(df2)*(2/5)
n_sample2_test = int(len(df_sample2)-n_sample2_train)

df_sample2_train, df_sample2_test = split_train_test(df_sample2, n_sample2_train)

x_sample2_train=df_sample2_train.drop('is_traffic', axis=1)
y_sample2_train = df_sample2_train["is_traffic"]

x_sample2_test=df_sample2_test.drop('is_traffic', axis=1)
y_sample2_test = df_sample2_test["is_traffic"]


df_sample2_train.shape, df_sample2_test.shape, x_sample2_train.shape, y_sample2_train.shape, x_sample2_test.shape, x_sample2_test.shape

In [ ]:
m2 = RandomForestClassifier(n_jobs=-1) 
m2.fit(x_sample2_train,y_sample2_train)
predictions = m2.predict(x_sample2_test)
score = round(accuracy_score(y_sample2_test, predictions), 3)
cm1 = cm(y_sample2_test, predictions)
sns.heatmap(cm1, annot=True, fmt=".0f")
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.title('Accuracy Score: {0}'.format(score), size = 15)
plt.show()

##### <font color='brown'/>**Hem AVG_SPEED'ı hem de VEHICLE_COUNT'ı atıp tüm veride çalıştırıyorum.** </font>

In [ ]:
df=df_c[model_list_real2]

In [ ]:
n_train = int(len(df)*(3/5)) # test setimin boyutu :len(df)*(2/5)
n_test = int(len(df)-n_train)

df_train, df_test = split_train_test(df, n_train)

In [ ]:
x_train = df_train.drop('is_traffic', axis=1)
y_train = df_train["is_traffic"]

x_test = df_test.drop('is_traffic', axis=1)
y_test = df_test["is_traffic"]

In [ ]:
m3 = RandomForestClassifier(n_jobs=-1) 
m3.fit(x_train,y_train)
predictions = m3.predict(x_test)
score = round(accuracy_score(y_test, predictions), 3)
cm1 = cm(y_test, predictions)
sns.heatmap(cm1, annot=True, fmt=".0f")
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.title('Accuracy Score: {0}'.format(score), size = 15)
plt.show()

##### <font color='brown'/>**Hem AVG_SPEED'i hem de VEHICLE COUNT'ı atıp tüm veriyle çalıştırdığımız modelin feature importance'ına bakıyoruz.** </font>

In [ ]:
def rf_feat_importance(m, df):
    return pd.DataFrame({'columns':df.columns, 'importance':m.feature_importances_}
                       ).sort_values('importance', ascending=False)

In [ ]:
feature_importance = rf_feat_importance(m3, x_train); 

In [ ]:
feature_importance[:20]

In [ ]:
feature_importance.plot('columns', 'importance', figsize=(10,6), legend=False);

#### <font color='green'/> **XGBoost** </font>

In [ ]:
import xgboost as xgb

##### <font color='brown'/>**Hem AVG_SPEED'i hem de Vehicle Count'ı attım. 100.000 veriyle çalıştırdım.** </font> 

In [ ]:
classifier = xgb.XGBClassifier(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=3
)

classifier.fit(x_sample2_train, y_sample2_train)

In [ ]:
predictions = classifier.predict(x_sample2_test)
score = round(accuracy_score(y_sample2_test, predictions), 3)
cm1 = cm(y_sample2_test, predictions)
sns.heatmap(cm1, annot=True, fmt=".0f")
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.title('Accuracy Score: {0}'.format(score), size = 15)
plt.show()

##### <font color='brown'/>**XGBOOST algoritmasını tüm veride çalıştırıyoruz.** </font> 

In [ ]:
classifier = xgb.XGBClassifier(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=3
)

classifier.fit(x_train, y_train)

In [ ]:
predictions = classifier.predict(x_test)
score = round(accuracy_score(y_test, predictions), 3)
cm1 = cm(y_test, predictions)
sns.heatmap(cm1, annot=True, fmt=".0f")
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.title('Accuracy Score: {0}'.format(score), size = 15)
plt.show()

#### <font color='green'/> **Logistic Regression**  </font>

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

##### <font color='brown'/>**Hem AVG_SPEED'i hem de Vehicle Count'ı attım. 100.000 veriyle çalıştırdım.** </font> 

In [ ]:
logreg = LogisticRegression()
logreg.fit(x_sample2_train, y_sample2_train)

In [ ]:
predictions = logreg.predict(x_sample2_test)
score = round(accuracy_score(y_sample2_test, predictions), 3)
cm1 = cm(y_sample2_test, predictions)
sns.heatmap(cm1, annot=True, fmt=".0f")
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.title('Accuracy Score: {0}'.format(score), size = 15)
plt.show()

##### <font color='brown'/>**Logistic Regression algoritmasını tüm veride çalıştırıyoruz.** </font> 

In [ ]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)

In [ ]:
predictions = logreg.predict(x_test)
score = round(accuracy_score(y_test, predictions), 3)
cm1 = cm(y_test, predictions)
sns.heatmap(cm1, annot=True, fmt=".0f")
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.title('Accuracy Score: {0}'.format(score), size = 15)
plt.show()

### <font color='blue'/> **7. y değerinin belirlenmesi** </font>

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=[8,7])
sns.distplot(data['AVG_SPEED']);  #ortalama hızın dağılımı(y ekseni : o hızın ne kadar görüldüğünü temsil ediyor)

In [ ]:

list = ['Year','Is_year_end'] # verimiz 2020 yılı ve yıl sonu olmadığı için heatmapte boş geliyordu bu featurelar. o yüzden çıkardık
x = data.drop(list,axis = 1 ) 


In [ ]:
f, ax = plt.subplots(figsize=(18, 18))
sns.heatmap(x.corr(), annot=True,linewidths=.5)  #annot=true her hücreye değerleri yazdırmak için
plt.show()


In [ ]:
#hız ile araç sayısı arasıdaki ilişki
plt.figure(figsize=(10,6))
plt.scatter(x=data['VEHICLE_COUNT'], y=data['AVG_SPEED'],c="#faa307ff", alpha=0.5)
plt.title('Araç Sayısına İlişkin Ortalama Hız', weight='bold', fontsize=15)

plt.xlabel('Araç Sayısı', weight='bold', fontsize=12)
plt.ylabel('Ortalama Hız', weight='bold', fontsize=12)
plt.xticks(weight='bold')
plt.yticks(weight='bold')
plt.show()


In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
type(data['Date'])

### <font color='blue'/> **8. Görselleştirme Çalışması** </font>

In [ ]:
ax=sns.countplot(td['ANNOUNCE_TYPE'],palette="RdYlGn") #veri setindeki tüm bildirimler
ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
g = sns.catplot(x="ANNOUNCE_TYPE",  col="COUNTY_NAME",  #ilçelere göre bildirimler
...                 data=td, kind="count",
...                 height=4, aspect=.9,palette="RdYlGn"); #"YlOrRd"
g.set_xticklabels(rotation=80)

#### <font color='green'/> **Trafik Yoğunluk Verisiyle Grafik Çalışmaları** </font>

In [ ]:
import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#%matplotlib qt
# pip install PyQt5

In [ ]:
df_traffic=pd.read_json("TRAFFIC_DENSITY.json",encoding = "UTF-8")

In [ ]:
df_traffic.head()

In [ ]:
df_traffic.sort_values(by=['DATE_TIME','GEOHASH6'],inplace=True)

In [ ]:
df_traffic.head()

In [ ]:
df_traffic[['year','month','day']] = df_traffic.DATE_TIME.apply(lambda x: pd.Series(
    x.strftime("%Y,%m,%d").split(","))) # split date into year, month, day

In [ ]:
df_traffic.head()

In [ ]:
d = df_traffic.groupby(['month']).mean()

In [ ]:
d.reset_index(inplace=True)
d

In [ ]:
df = pd.DataFrame(d) 
  
# Declare a list that is to be converted into a column 
Month = ['Ocak', 'Şubat', 'Mart', 'Nisan', "Mayıs", "Haziran"] 
  
df['Aylar'] = Month 
 
df 

In [ ]:
sns.distplot(d.AVG_SPEED); #yoğunluk grafiği

In [ ]:
from matplotlib import pyplot

In [ ]:
pyplot.figure(figsize=(10,5))
plt.plot(d.Aylar,d.AVG_SPEED, "#E85D04", linewidth=5.0, label='Ortalama Hız')
plt.title("Aylara Göre Ortalama Araç Hızları")
plt.xlabel("Aylar")
plt.legend(loc='lower left')
plt.grid(False)
plt.ylabel("Ortalama Araç Hızları");

In [ ]:
pyplot.figure(figsize=(10,5))
pyplot.plot(d.Aylar,d.VEHICLE_COUNT,"#DC2F02", linewidth=7.0, linestyle="--", label='Araç Sayısı')
pyplot.title('Aylara Göre Ortalama Araç Sayısı')
plt.legend(loc='lower left')
plt.grid(False)
pyplot.xlabel('Aylar');
plt.ylabel("Ortalama Araç Sayıları");

In [ ]:
pyplot.figure(figsize=(10,5))
pyplot.plot(d.Aylar,d.VEHICLE_COUNT,"#DC2F02", linewidth=7.0, linestyle="--", label='Araç Sayısı')
pyplot.title('Aylara Göre Ortalama Araç Sayısı')
plt.legend(loc='lower left')
plt.grid(False)
pyplot.xlabel('Aylar');
plt.ylabel("Ortalama Araç Sayıları");

#### <font color='green'/> **TKM Verisindeki Kooordinat Verisinden Yararlanılarak Harita Görselleştirme Çalışması** </font>

In [ ]:
import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot
%matplotlib inline
#%matplotlib qt
# pip install PyQt5

In [ ]:
df_tkm =pd.read_excel("TKM.xlsx",encoding = "UTF-8")
df_tkm.head()

In [ ]:
df_tkm.drop(["Unnamed: 0","COUNTY_NAME","GEOHASH_CHECK","TRAFIK_DENSITY_Geohash"], axis=1, inplace=True)  

In [ ]:
df_tkm.head()

In [ ]:
df_tkm[['year','month','day']] = df_tkm.ANNOUNCE_TIME.apply(lambda x: pd.Series(
    x.strftime("%Y,%m,%d").split(","))) # split date into year, month, day

In [ ]:
df_tkm.head()

In [ ]:
!pip install folium 
#conda install -c conda-forge folium
import folium

In [ ]:
from folium import plugins
m = folium.Map(location=[41,29],
tiles="cartodbpositron",
 width="%100",
height="%100",
zoom_start=10)
 
minimap = plugins.MiniMap()
m.add_child(minimap)
m

In [ ]:
locations = list(zip(df_tkm.ANNOUNCE_LATITUDE, df_tkm.ANNOUNCE_LONGTITUDE))

In [ ]:
m = folium.Map(location=[41,29],
tiles="cartodbpositron",
 width="%100",
height="%100",
zoom_start=10)
for i in range(len(locations)):
    folium.CircleMarker(location=locations[i],radius=1, color='#FF7D40',).add_to(m)
m

In [ ]:
# convert to (n, 2) nd-array format for heatmap
stationArr = df_tkm[['ANNOUNCE_LATITUDE', 'ANNOUNCE_LONGTITUDE']].as_matrix()

# plot heatmap
m.add_child(plugins.HeatMap(stationArr, radius=15))
m 

m.add_child(folium.LayerControl())
m.save('heatmap.html')

In [ ]:
m1 = folium.Map(location=[41,29],
tiles="cartodbpositron",
width="%100",
height="%100",
legend_name='Trafik Duyurular',
zoom_start=10)

cluster = plugins.MarkerCluster(locations=locations,                     
               popups=df_tkm["ANNOUNCE_TYPE"].tolist(), encoding="UTF-8")  
m1.add_child(cluster)
m1
m1.add_child(folium.LayerControl())
m1.save(outfile='duyurular.html')